In [1]:
%%capture 
!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install scipy 
!pip install statsmodels
!pip install matplotlib
!pip install gdal
!pip install numpy
!pip install scikit-learn
!pip install statsmodels

In [2]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import scipy.stats as stats
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
def get_val_in_fire_canlabs(fire_shapefile, shapefile,data_surface,\
                            transform,size,srcds,year_fire,fire_years_raster,\
                            factor,threshold=0.8):
    '''This is a function to get the value inside the fire.
    We will use to calculate the mean, median, max value for a fire.
    
    Parameters
    ----------
        fire_shapefile : string
            path to the fire shapefile 
        shapefile : string
            path to the study area shapefile
        data_surface : ndarray
            an array of values in the study area
        transform : list 
            list describing GeoTransform of raster 
        size : list 
            pixel dimensions
        srcds : GDAL object
            read in raster
        year_fire : int
            year of fire from CNFDB
        fire_years_raster : ndarray 
            fire years raster from canlabs in ndarray format
        factor : int
            factor by which to multiply number of points in each pixel 
            higher = more accurate 
        threshold : float
            how much cloud mask to tolerate      
            by default it is 0.8 
            
    Returns
    ----------
        float
            - maximum value in fire, either the closest point to the convex hull of the fire or a sum of the points inside the fire
    '''

    #na_map = gpd.read_file(shapefile)
    #bounds = shapefile.bounds  # Get the bounding box of the shapefile
    bounds = gpd.GeoDataFrame(geometry=[fire_shapefile['geometry']]).bounds 
    xmax = np.nanmax(bounds['maxx'])
    xmin = np.nanmin(bounds['minx'])
    ymax = np.nanmax(bounds['maxy'])
    ymin = np.nanmin(bounds['miny'])

    xOrigin = transform[0]
    yOrigin = transform[3]
    xMax = xOrigin + transform[1] * size[0]
    yMin = yOrigin + transform[5] * size[1]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    # Calculate the number of rows cols to fill the bounding box at that resolution
    num_col = int((xmax - xmin) / pixelHeight)+1
    num_row = int((ymax - ymin) / pixelWidth)+1

    # Add the bounding box coords to the dataset so we can extrapolate the interpolation to cover whole area
    yProj_extent = [bounds['maxy'], bounds['miny']]
    xProj_extent = [bounds['maxx'], bounds['minx']]

    # Get the value for lat lon in each cell we just made
    Yi = np.linspace(np.min(yProj_extent), np.max(yProj_extent), num_row*factor)
    Xi = np.linspace(np.min(xProj_extent), np.max(xProj_extent), num_col*factor)

    Xi, Yi = np.meshgrid(Xi, Yi)
    # Because we are not using the lookup file, send in X,Y order
    concat = np.array((Xi.flatten(), Yi.flatten())).T
    send_to_list = concat.tolist()

    meshPoints = [Point(item) for item in send_to_list]
    #df = pd.DataFrame(meshPoints)
    gdf = gpd.GeoDataFrame(geometry=meshPoints)

    DF = fire_shapefile
    try: #If there is a single polygon in the shapefile
        DF = unary_union(Polygon(DF['geometry'])) #Multipolygon --> Polygon
        poly_define = gpd.GeoDataFrame(geometry=[DF])
        # Get points falling in fire 
        within_fire = gdf[gdf.geometry.within(poly_define['geometry'][0])]
    # Catch the case where multiple polygons not continuous make up the shp
    except (NotImplementedError,AttributeError,TypeError) as e:
        DF = [unary_union(Polygon(geom)) for geom in list(DF['geometry'])]
        poly_define = gpd.GeoDataFrame(geometry=DF)
        # Left spatial join 
        within_fire = sjoin(gdf, poly_define, how='left',op='within')
        # Drop points that are not in the fire 
        within_fire = within_fire[~np.isnan(within_fire['index_right'])]


    inside_fire = []
    years_inside_fire = [] 
    lon = []
    lat = [] 
    listP = within_fire

    
    for idx,p in listP.iterrows():
        #print(p)
        mx,my=np.array(p['geometry'].coords.xy[0])[0], np.array(p['geometry'].coords.xy[1])[0]
        
        col = int(np.floor((mx - xOrigin)) / pixelHeight)
        row = int(np.floor((my - yOrigin)) / pixelWidth)

        try: 
            sev = data_surface[row-1][col]
        except IndexError:
            sev= -32768
            
        if sev != -32768: #No data value
            inside_fire.append(data_surface[row-1][col])
            years_inside_fire.append(fire_years_raster[row-1][col])
            
        else:
            inside_fire.append(np.nan)
            years_inside_fire.append(np.nan)
        
        lon.append(mx)
        lat.append(my)
        
    
    within_fire['max_val'] = inside_fire
    within_fire['lon'] = lon
    within_fire['lat'] = lat
    within_fire['years'] = years_inside_fire


    within_fire = within_fire[within_fire['years'] == year_fire]

    if len(within_fire) > 0:
        numnan = len(within_fire[~np.isnan(within_fire['max_val'])]) / len(within_fire['max_val'])
        print(numnan)

        if numnan >= threshold:
            mean = np.nanmean(inside_fire)  # get the mean val inside the fire
            median = np.nanmedian(inside_fire)
            maximum = np.nanmax(inside_fire)

        else:
            mean = np.nan
            median = np.nan
            maximum = np.nan
            
    else:
        mean = np.nan
        median = np.nan
        maximum = np.nan

    return mean,median,maximum   

In [4]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Mounted at /content/drive
/content/drive/MyDrive/jameslab


In [ ]:
sa = gpd.read_file('study_area/BC_AB_shapefile.shp')
shp = gpd.read_file('fire_perimeters/NFDB_poly_20210707_epsg4326.shp')
shp = shp[shp['SIZE_HA'] >= 10]
shp_prov = shp[shp['SRC_AGENCY'].isin(['AB','BC'])]
shp_year = shp_prov[shp_prov['YEAR'] >= 1985]
shp_year = shp_year[shp_year['YEAR'] <= 2015]
shp_type = shp_year[shp_year['CAUSE'] == 'L'].to_crs('esri:102001')
print(len(shp_type))

fire_sev_list = []
y_ds1 = gdal.Open('canlabs/years_BC_AB.tif')
yb2=y_ds1.GetRasterBand(1)
ycols = y_ds1.RasterXSize
yrows = y_ds1.RasterYSize


ydata = yb2.ReadAsArray(0, 0, ycols, yrows)

src_ds1 = gdal.Open('canlabs/canlabs_BC_AB.tif')
rb2=src_ds1.GetRasterBand(1)
transform=src_ds1.GetGeoTransform()
cols = src_ds1.RasterXSize
rows = src_ds1.RasterYSize

data = rb2.ReadAsArray(0, 0, cols, rows)

for index, fire in shp_type.iterrows():
    print(fire['FIRE_ID'])
    year = int(fire['YEAR'])

    ave,med,max_val = get_val_in_fire_canlabs(fire, sa,data,transform,\
                                              (cols,rows,),src_ds1,year,\
                                              ydata,1,threshold=0.8)
    fire_sev_list.append([ave,med,max_val])

shp_type['fire_severity_mean'] = [x[0] for x in fire_sev_list]
shp_type['fire_severity_median'] = [x[1] for x in fire_sev_list]
shp_type['fire_severity_max'] = [x[2] for x in fire_sev_list]
print(shp_type)

shp_type.to_csv('fire_severity_in_cnfdb_fires.txt',sep=',')

2402
2004-C10175
2004-C10176
2004-C50114
2004-C50125
2004-C50149
2004-C50265
2004-C50508
2004-G30401
2004-G30485
2004-G40152
2004-G40189
2004-G40496
2004-G50165
2004-G60166
2004-G80200
2004-G80238
2004-G90138
2004-G90232
2004-G90272
2004-G90273
2004-G90310
2004-G90352
2004-G90440
2004-G90441
2004-K10367
2004-K10370
2004-K10384
2004-K10398
2004-K10409
2004-K10413
2004-K10413
2004-K10422
2004-K10426
2004-K10429
2004-K10442
2004-K20139
2004-K40563
2004-K60610
2004-K70099
2004-K70112
2004-K70172
2004-K70198
2004-K70498
2004-K70499
2004-N40184
2004-N40379
2004-N70177
2004-N70398
2004-N70493
2004-N70493
2004-R10065
2004-R10115
2004-R10243
2004-R20052
2004-R20139
2004-R20148
2004-R30046
2004-R40111
2004-R40142
2004-R40163
2004-R50048
2004-R50068
2004-R50071
2004-R50121
2004-R90045
2004-R90083
2004-R90086
2004-R90087
2004-R90097
2004-R90101
2004-R90106
2004-R90134
2004-R90182
2004-R90183
2004-R90184
2004-R90189
2004-R90198
2004-R90214
2004-R90236
2004-V10531
2004-V10543
2004-VA0194
2005-C10054